In [18]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.metrics import classification_report

In [19]:
#Pre-Processing - Complete
import io
import shutil

with io.open('Recent Pull.txt', encoding='utf-8', errors='ignore') as source:
    with io.open('Recent Pull_b.txt', mode='w', encoding='utf-8') as target:
        shutil.copyfileobj(source, target)

In [20]:
test_grant = pd.read_table('Recent Pull_b.txt',error_bad_lines=False)
data_conc = test_grant['Grant Title'].map(str) + " " + test_grant['Description'].map(str) 
test_grant['Data_Conc'] = data_conc
test_grant

,GID,Grant Title,Description,Program Purpose,Information_on_Eligibility,Eligibility Code(s),Grant Category,Grantor Type,Creation Date,Data_Conc
0,106988,Ambassadors Fund for Cultural Preservation 202...,The U.S. Embassy in Chisinau is pleased to ann...,NaN,Applicants must be non-governmental organizati...,25,O,5.0,2019-10-25,Ambassadors Fund for Cultural Preservation 202...
1,106989,2020-DRAFT-IFRP-PRODUCT-RFA-DRAFT,DRAFT IFRP PRODUCT RFA. The United States Agen...,NaN,NaN,12,FN,5.0,2019-10-25,2020-DRAFT-IFRP-PRODUCT-RFA-DRAFT DRAFT IFRP P...
2,106990,FY 2020 DRL Internet Freedom Annual Program St...,"The Bureau of Democracy, Human Rights, and Lab...",NaN,Please see &quot;Eligibility Information&quot;...,25,O,5.0,2019-10-25,FY 2020 DRL Internet Freedom Annual Program St...
3,106991,U.S. Embassy Managua - Annual Program Statement,The U.S. Embassy Public Affairs Section - Smal...,NaN,Eligible applicants for this Annual Program St...,"12, 13, 21, 25","CD, ED, O, ELT",5.0,2019-10-25,U.S. Embassy Managua - Annual Program Stateme...
4,106991,U.S. Embassy Managua - Annual Program Statement,The U.S. Embassy Public Affairs Section - Smal...,NaN,Eligible applicants for this Annual Program St...,"12, 13, 21, 25","CD, ED, O, ELT",5.0,2019-10-25,U.S. Embassy Managua - Annual Program Stateme...
...,...,...,...,...,...,...,...,...,...,...
353,107251,Strengthening Multi-sectoral Coordination and ...,The Award Ceiling for Year 1 is 0 (none). CDC ...,NaN,Eligible applicants that can apply for this NO...,25,HL,5.0,2019-11-19,Strengthening Multi-sectoral Coordination and ...
354,107252,FY 20-21 Fishery Management Council Coral Reef...,The NOAA Coral Reef Conservation Grant Program...,NaN,Eligible applicants are limited to the Caribbe...,25,"ENV, NR, ST",5.0,2019-11-19,FY 20-21 Fishery Management Council Coral Reef...
355,107252,FY 20-21 Fishery Management Council Coral Reef...,The NOAA Coral Reef Conservation Grant Program...,NaN,Eligible applicants are limited to the Caribbe...,25,"ENV, NR, ST",5.0,2019-11-19,FY 20-21 Fishery Management Council Coral Reef...
356,107252,FY 20-21 Fishery Management Council Coral Reef...,The NOAA Coral Reef Conservation Grant Program...,NaN,Eligible applicants are limited to the Caribbe...,25,"ENV, NR, ST",5.0,2019-11-19,FY 20-21 Fishery Management Council Coral Reef...


In [21]:
#Filter to Grant Category
categories = {'AG': ('AG.csv', [3,4,5]),'AR': ('AR.csv', [2]),'BC': ('BC.csv', [0,2,3]),'CD': ('CD.csv', [1,3]),
              'CH': ('CH.csv', [2]),'ED': ('ED.csv', [0,2, 3, 7, 9, 10]),'ELT': ('ELT.csv', [2]),'EN': ('EN.csv', [3,6]),
              'ENV': ('ENV.csv', [2, 4]),'FN': ('FN.csv', [1]),'HL': ('HL.csv', [9, 14, 15]),'HO': ('HO.csv', [1, 2]),
              'HU': ('HU.csv', [1, 2]),'IS': ('IS.csv', [0, 3]),'ISS': ('ISS.csv', [1, 5]),'LJL': ('LJL.csv', [0, 2, 3]),
              'NR': ('NR.csv', [2, 4]),'RD': ('RD.csv', [0, 4]),'SC': ('SC.csv', [3]),'ST': ('ST.csv', [1, 2, 3, 5, 6]),
              'STU': ('STU.csv', [1, 3, 6]),'T': ('T.csv', [0, 2, 3])}

def getList(dibt): 
    
    return list(dibt.keys()) 

category_list = getList(categories)

category_list.sort()

print(category_list)

['AG', 'AR', 'BC', 'CD', 'CH', 'ED', 'ELT', 'EN', 'ENV', 'FN', 'HL', 'HO', 'HU', 'IS', 'ISS', 'LJL', 'NR', 'RD', 'SC', 'ST', 'STU', 'T']


In [28]:
# Instantiate Model - Basically done
for cat in category_list:
    pred_grant = pd.read_csv("Training Data/" + categories[cat][0])
    pred_grant.drop(columns=['Unnamed: 0'], inplace=True)
    viable_cluster = categories[cat][1]

    for cluster in list(pred_grant.Category.drop_duplicates()):
        labels = []
        name = ('is_cluster_%d' %(cluster))
        for label in pred_grant.Category:
            if label == cluster:
                labels.append(1)
            else:
                labels.append(0)

        pred_grant[name] = labels

    for cluster in viable_cluster:
        # define X and y
        column = 'is_cluster_' + str(cluster)
        X_Train = pred_grant.Data_Conc
        X_Pred = test_grant.Data_Conc
        Y = pred_grant[column]

        #vectorizer for title text
        vect = CountVectorizer()
            #ngram_range=(1,3), stop_words = 'english'

        # create document-term matrices
        X_Train_dtm = vect.fit_transform(X_Train)
        X_Pred_dtm = vect.transform(X_Pred)

        #instantiate logistic regression model
        log = LogisticRegression()
        log.fit(X_Train_dtm, Y)
        Pred_Class = log.predict(X_Pred_dtm)

# Label the Fresh File - Need to finish
        col_name = ('%s_%d' %(cat,cluster))
        test_grant[col_name] = Pred_Class

C:\Users\assef\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\assef\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\assef\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\assef\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\assef\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence t

C:\Users\assef\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\assef\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\assef\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\assef\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\assef\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence t

In [33]:
clean_test = test_grant.copy()

del clean_test['Program Purpose']
del clean_test['Information_on_Eligibility']
del clean_test['Grantor Type']
del clean_test['Creation Date']
del clean_test['Grant Title']
del clean_test['Description']
del clean_test['Eligibility Code(s)']

clean_test

,GID,Grant Category,Data_Conc,AG_3,AG_4,AG_5,AR_2,BC_0,BC_2,BC_3,...,ST_2,ST_3,ST_5,ST_6,STU_1,STU_3,STU_6,T_0,T_2,T_3
0,106988,O,Ambassadors Fund for Cultural Preservation 202...,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,106989,FN,2020-DRAFT-IFRP-PRODUCT-RFA-DRAFT DRAFT IFRP P...,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2,106990,O,FY 2020 DRL Internet Freedom Annual Program St...,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,1,0
3,106991,"CD, ED, O, ELT",U.S. Embassy Managua - Annual Program Stateme...,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
4,106991,"CD, ED, O, ELT",U.S. Embassy Managua - Annual Program Stateme...,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
353,107251,HL,Strengthening Multi-sectoral Coordination and ...,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
354,107252,"ENV, NR, ST",FY 20-21 Fishery Management Council Coral Reef...,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
355,107252,"ENV, NR, ST",FY 20-21 Fishery Management Council Coral Reef...,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
356,107252,"ENV, NR, ST",FY 20-21 Fishery Management Council Coral Reef...,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0


In [36]:
clean_test.to_csv("Newly Labeled Data.csv")